In [1]:
import pandas as pd

df = pd.read_csv('ecommerce_customer_data_large.csv')

In [7]:
df.Churn.value_counts()

0    199870
1     50130
Name: Churn, dtype: int64

In [8]:
df.Returns.value_counts()

1.0    101476
0.0    101142
Name: Returns, dtype: int64

In [2]:
import pandas as pd

customer_info_columns = ['Customer ID', 'Customer Name', 'Customer Age', 'Age', 'Gender', 'Churn']
customers = df[customer_info_columns].drop_duplicates().reset_index(drop=True)

purchase_info_columns = ['Customer ID', 'Purchase Date', 'Product Category', 'Product Price', 'Quantity', 'Total Purchase Amount', 'Payment Method', 'Returns']
transations = df[purchase_info_columns]


In [6]:
import numpy as np

transations['Transaction ID'] = np.arange(start=100, stop=100 + len(transations))


/var/folders/j2/_fwtsnt17yv2984h526c70x40000gn/T/ipykernel_71351/3832575821.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transations['Transaction ID'] = np.arange(start=100, stop=100 + len(transations))


In [15]:
customers.to_csv('customers.csv', index=False)

In [7]:
transations.to_csv('transations.csv', index=False)

In [ ]:
import pandas as pd
from datetime import datetime

customers_df = pd.read_csv('customers_clean.csv')
transactions_df = pd.read_csv('transactions_clean.csv')
transactions_df['Purchase_Date'] = pd.to_datetime(transactions_df['Purchase_Date'])

# calculate days since last purchase for each customer
reference_date = max(transactions_df['Purchase_Date'].max(), datetime.now())
last_purchase_df = transactions_df.groupby('Customer_ID')['Purchase_Date'].max().reset_index()
last_purchase_df['Days_Since_Last_Purchase'] = (reference_date - last_purchase_df['Purchase_Date']).dt.days
last_purchase_df = last_purchase_df[['Customer_ID', 'Days_Since_Last_Purchase']]

# calculate total spend estimate (CLTV) for each customer
cltv_df = transactions_df.groupby('Customer_ID')['Total_Purchase_Amount'].sum().reset_index()
cltv_df.columns = ['Customer_ID', 'CLTV']

# calculate average transaction value for each customer
avg_txn_value_df = transactions_df.groupby('Customer_ID')['Total_Purchase_Amount'].mean().reset_index()
avg_txn_value_df.columns = ['Customer_ID', 'Avg_Txn_Value']

# favorite category for each customer
fav_category_df = transactions_df.groupby(['Customer_ID', 'Product_Category']).size().reset_index(name='count')
fav_category_df = fav_category_df.sort_values(['Customer_ID', 'count'], ascending=[True, False])
fav_category_df = fav_category_df.drop_duplicates(subset=['Customer_ID'])
fav_category_df.columns = ['Customer_ID', 'Fav_Category', 'count']

# return rate for each customer
returns_df = transactions_df.groupby('Customer_ID')['Returns'].sum().reset_index()
total_txn_df = transactions_df.groupby('Customer_ID').size().reset_index(name='total_transactions')
return_rate_df = pd.merge(returns_df, total_txn_df, on='Customer_ID')
return_rate_df['Return_Rate'] = return_rate_df['Returns'] / return_rate_df['total_transactions']
return_rate_df = return_rate_df[['Customer_ID', 'Return_Rate']]

# favorite payment method for each customer
fav_payment_method_df = transactions_df.groupby(['Customer_ID', 'Payment_Method']).size().reset_index(name='count')
fav_payment_method_df = fav_payment_method_df.sort_values(['Customer_ID', 'count'], ascending=[True, False])
fav_payment_method_df = fav_payment_method_df.drop_duplicates(subset=['Customer_ID'])
fav_payment_method_df.columns = ['Customer_ID', 'Fav_Payment_Method', 'count']

# merge all data
churn_analysis_df = customers_df.merge(last_purchase_df, on='Customer_ID', how='left')
churn_analysis_df = churn_analysis_df.merge(cltv_df, on='Customer_ID', how='left')
churn_analysis_df = churn_analysis_df.merge(avg_txn_value_df, on='Customer_ID', how='left')
churn_analysis_df = churn_analysis_df.merge(fav_category_df[['Customer_ID', 'Fav_Category']], on='Customer_ID', how='left')
churn_analysis_df = churn_analysis_df.merge(return_rate_df, on='Customer_ID', how='left')
churn_analysis_df = churn_analysis_df.merge(fav_payment_method_df[['Customer_ID', 'Fav_Payment_Method']], on='Customer_ID', how='left')
churn_analysis_df = churn_analysis_df[['Customer_ID', 'Age', 'Gender', 'Days_Since_Last_Purchase', 'CLTV', 'Avg_Txn_Value', 'Fav_Category', 'Return_Rate', 'Fav_Payment_Method', 'Churn']]

churn_analysis_df.to_csv('churn_analysis.csv', index=False)

In [11]:
churn_analysis_df.Return_Rate.value_counts()

0.500000    8156
0.000000    6121
0.333333    5826
0.250000    3610
0.400000    3346
            ... 
0.769231       1
0.235294       1
0.900000       1
0.076923       1
0.437500       1
Name: Return_Rate, Length: 64, dtype: int64